# Algorytm przeszukiwania Tabu (TS)

## Biblioteki

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
from collections import deque
from openpyxl import load_workbook

## Pobór danych 

In [2]:
file_path1 = "./Dane_TSP_48.xlsx"
file_path2 = "./Dane_TSP_76.xlsx"
file_path3 = "./Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()
data2 = pd.read_excel(file_path2, index_col=0).to_numpy()
data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Rodzaje ruchów

In [3]:
# Wybieramy dwie pozycje i zmieniamy je miejscami 
def swap_move(path):  
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:]
        new_path[i], new_path[j] = new_path[j], new_path[i]
        yield new_path
 

# Odwracamy segment pomiędzy wybranymi wartościami czyli 
# [a b c d e] -> [a d c b e]
def two_opt_move(path):
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:i] + path[i:j][::-1] + path[j:]
        yield new_path
 
# Wybieramy jeden wierzchołek w ścieżce, usuwamy z jednego miejsca i przerzucamy w inne miejsce
def insertion_move(path):
    n = len(path)
    for i in range(n):
        for j in range(n):
            if i != j:
                new_path = path[:]
                node = new_path.pop(i)
                new_path.insert(j, node)
                yield new_path


## Długość ścieżki

In [4]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Implementacja algorytmu

In [5]:
def tabu_search(distance_matrix, tabu_list_length=10, max_iterations=50, max_no_improve=1000, move_type="two_opt", time_limit=1000):
    start_time = time.time()
    
    n = len(distance_matrix)
    
    # initial solution
    best_solution = list(range(n))
    random.shuffle(best_solution)
    best_cost = calculate_path_cost(distance_matrix, best_solution) # Obliczamy koszt początkowego rozwiązania
    current_solution = best_solution[:] # Aktualne rozwiązanie to początkowe
    current_cost = best_cost

    # Inicjalizujemy listę tabu z ograniczoną długością
    tabu_list = deque(maxlen=tabu_list_length)
    no_improve = 0 # Licznik iteracji bez poprawy
    best_time = 0  # Zmienna do przechowywania czasu, w którym znaleziono najlepsze rozwiązanie
 
    move_generators = {
        "swap": swap_move,
        "two_opt": two_opt_move,
        "insertion": insertion_move
    }
 
    if move_type not in move_generators:
        raise ValueError(f"Nieznany typ ruchu: {move_type}")
 
    move_generator = move_generators[move_type]
 
    for iteration in range(max_iterations):
        if time_limit and (time.time() - start_time > time_limit):
            print("Przekroczono limit czasu.")
            break
 
        # Generujemy sąsiedztwo dla bieżącego rozwiązania
        neighborhood = list(move_generator(current_solution))
        neighborhood_costs = [calculate_path_cost(distance_matrix, neighbor) for neighbor in neighborhood]
 
        best_neighbor = None # Najlepszy sąsiad
        best_neighbor_cost = float('inf')
 
        # Przeszukujemy sąsiedztwo, aby znaleźć najlepszy sąsiadujący ruch, który nie jest w liście tabu
        for neighbor, cost in zip(neighborhood, neighborhood_costs):
            if (neighbor not in tabu_list) and (cost < best_neighbor_cost):
                best_neighbor = neighbor
                best_neighbor_cost = cost
 
        if best_neighbor is None:
            break
 
        # Update current solution and tabu list
        current_solution = best_neighbor
        current_cost = best_neighbor_cost
        tabu_list.append(current_solution)
        
        # Update best solution if improved
        if current_cost < best_cost:
            best_solution = current_solution[:]
            best_time = time.time() - start_time
            best_cost = current_cost
            no_improve = 0 # Resetujemy licznik iteracji bez poprawy
        else:
            no_improve += 1

        # Jeśli osiągnięto limit iteracji bez poprawy, kończymy algorytm
        if no_improve >= max_no_improve:
            print("Osiągnięto limit iteracji bez poprawy.")
            break

        if iteration % 10 == 0:
            elapsed_time = time.time() - start_time
            print(f"Iteration {iteration}, Best Cost: {best_cost:.2f}, Elapsed Time: {elapsed_time:.2f} seconds")
 
    return best_solution, best_cost, best_time

## Generowanie wyników

Za podstawowe dane przyjmujemy:

* tabu_list_length = 10

* max_iterations = 10000

* max_no_improve = 50

* move_type = `two_opt`

* time_limit = 1000

Następnie badane są wpływy zmiany wartości poszczególnych parametrów 

In [6]:
# Dane_TSP_48
best_path_1, best_cost_1, best_time_1 = tabu_search(data1)
# Dane_TSP_76
best_path_2, best_cost_2, best_time_2 = tabu_search(data2)
# Dane_TSP_127
best_path_3, best_cost_3, best_time_3 = tabu_search(data3)

Iteration 0, Best Cost: 48221.00, Elapsed Time: 0.02 seconds
Iteration 10, Best Cost: 24307.00, Elapsed Time: 0.18 seconds
Iteration 20, Best Cost: 15992.00, Elapsed Time: 0.37 seconds
Iteration 30, Best Cost: 12514.00, Elapsed Time: 0.57 seconds
Iteration 40, Best Cost: 11591.00, Elapsed Time: 0.77 seconds
Iteration 0, Best Cost: 547693.67, Elapsed Time: 0.07 seconds
Iteration 10, Best Cost: 380194.23, Elapsed Time: 0.88 seconds
Iteration 20, Best Cost: 270155.43, Elapsed Time: 1.60 seconds
Iteration 30, Best Cost: 192811.19, Elapsed Time: 2.34 seconds
Iteration 40, Best Cost: 153515.73, Elapsed Time: 3.02 seconds
Iteration 0, Best Cost: 630368.25, Elapsed Time: 0.29 seconds
Iteration 10, Best Cost: 488807.15, Elapsed Time: 3.18 seconds
Iteration 20, Best Cost: 385966.63, Elapsed Time: 5.78 seconds
Iteration 30, Best Cost: 307801.24, Elapsed Time: 8.66 seconds
Iteration 40, Best Cost: 252584.91, Elapsed Time: 11.55 seconds


In [7]:
W1 =  {
    "Długość ścieżki": [best_cost_1, best_cost_2, best_cost_3],
    "Ścieżka": [best_path_1, best_path_2, best_path_3],
    "Czas": [best_time_1, best_time_2, best_time_3]
}

basic = pd.DataFrame(data = W1)

with pd.ExcelWriter('TS.xlsx', engine='openpyxl', mode='w') as writer:
    basic.to_excel(writer, sheet_name = "Basic", index=False)

### Badanie wpływu zmian wartości parametru `tabu_list_length`

In [8]:
def test_tabu_list_length(data, dataset_name, tabu_tendures, num_repeats=10):

    results = []
    for tabu_list_length in tabu_tendures:
        for _ in range(num_repeats):
            best_solution, best_cost, best_time = tabu_search(
                data,
                tabu_list_length=tabu_list_length,
                max_iterations=10000,
                max_no_improve=50,
                move_type="two_opt",
                time_limit=1000
            )
            # Zbieranie wyników
            results.append({
                "PARAMETR": tabu_list_length,
                "WYNIK_1": best_cost,
                "ŚCIEŻKA": best_solution,
                "CZAS": best_time
            })
    # Tworzenie DataFrame z wynikami
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Lista długości tabu do przetestowania
tabu_tendures = [5, 10, 15, 20]

# Testowanie dla trzech zestawów danych
df_tabu_list_1 = test_tabu_list_length(data1, "DATA1", tabu_tendures)
df_tabu_list_2 = test_tabu_list_length(data2, "DATA2", tabu_tendures)
df_tabu_list_3 = test_tabu_list_length(data3, "DATA3", tabu_tendures)

# Łączenie wyników w jeden DataFrame
tabu_results = pd.concat([df_tabu_list_1, df_tabu_list_2, df_tabu_list_3], ignore_index=True)

# Zapis wyników do pliku Excel
with pd.ExcelWriter('TS.xlsx', engine='openpyxl', mode='a') as writer:
    tabu_results.to_excel(writer, sheet_name='tabu_list_length', index=False)


Iteration 0, Best Cost: 48018.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 23397.00, Elapsed Time: 0.16 seconds
Iteration 20, Best Cost: 14899.00, Elapsed Time: 0.27 seconds
Iteration 30, Best Cost: 12084.00, Elapsed Time: 0.42 seconds
Iteration 40, Best Cost: 11159.00, Elapsed Time: 0.53 seconds
Iteration 50, Best Cost: 11095.00, Elapsed Time: 0.64 seconds
Iteration 60, Best Cost: 11095.00, Elapsed Time: 0.76 seconds
Iteration 70, Best Cost: 11095.00, Elapsed Time: 0.86 seconds
Iteration 80, Best Cost: 11095.00, Elapsed Time: 0.97 seconds
Iteration 90, Best Cost: 11095.00, Elapsed Time: 1.08 seconds
Osiągnięto limit iteracji bez poprawy.
Iteration 0, Best Cost: 46776.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 24221.00, Elapsed Time: 0.12 seconds
Iteration 20, Best Cost: 15846.00, Elapsed Time: 0.23 seconds
Iteration 30, Best Cost: 12862.00, Elapsed Time: 0.34 seconds
Iteration 40, Best Cost: 11861.00, Elapsed Time: 0.46 seconds
Iteration 50, Best Cost: 11587.00

### Badanie wpływu zmian wartości parametru `max_iterations`

In [9]:
def test_max_iterations(data, dataset_name, max_iterations_values, num_repeats=10):

    results = []
    for max_iterations in max_iterations_values:
        for _ in range(num_repeats):
            best_solution, best_cost, best_time = tabu_search(
                data,
                max_iterations=max_iterations,
                max_no_improve=50,
                move_type="two_opt",
                time_limit=1000,
                tabu_list_length=10
            )
            # Zbieranie wyników
            results.append({
                "PARAMETR": max_iterations,
                "WYNIK_1": best_cost,
                "ŚCIEŻKA": best_solution,
                "CZAS": best_time
            })
    # Tworzenie DataFrame z wynikami
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Lista wartości max_iterations do przetestowania
max_iterations_values = [500, 1000, 1500, 2500]

# Testowanie dla trzech zestawów danych
df_max_iterations_1 = test_max_iterations(data1, "DATA1", max_iterations_values)
df_max_iterations_2 = test_max_iterations(data2, "DATA2", max_iterations_values)
df_max_iterations_3 = test_max_iterations(data3, "DATA3", max_iterations_values)

# Łączenie wyników w jeden DataFrame
max_iterations_results = pd.concat([df_max_iterations_1, df_max_iterations_2, df_max_iterations_3], ignore_index=True)

# Zapis wyników do pliku Excel
with pd.ExcelWriter('TS.xlsx', engine='openpyxl', mode='a') as writer:
    max_iterations_results.to_excel(writer, sheet_name='max_iterations', index=False)

Iteration 0, Best Cost: 47533.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 24997.00, Elapsed Time: 0.14 seconds
Iteration 20, Best Cost: 15354.00, Elapsed Time: 0.26 seconds
Iteration 30, Best Cost: 12404.00, Elapsed Time: 0.44 seconds
Iteration 40, Best Cost: 11314.00, Elapsed Time: 0.61 seconds
Iteration 50, Best Cost: 11161.00, Elapsed Time: 0.82 seconds
Iteration 60, Best Cost: 11161.00, Elapsed Time: 1.00 seconds
Iteration 70, Best Cost: 11161.00, Elapsed Time: 1.15 seconds
Iteration 80, Best Cost: 11161.00, Elapsed Time: 1.32 seconds
Iteration 90, Best Cost: 11161.00, Elapsed Time: 1.52 seconds
Osiągnięto limit iteracji bez poprawy.
Iteration 0, Best Cost: 51422.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 25696.00, Elapsed Time: 0.19 seconds
Iteration 20, Best Cost: 15944.00, Elapsed Time: 0.34 seconds
Iteration 30, Best Cost: 12427.00, Elapsed Time: 0.45 seconds
Iteration 40, Best Cost: 11310.00, Elapsed Time: 0.57 seconds
Iteration 50, Best Cost: 11090.00

### Badanie wpływu zmian wartości parametru `max_no_improve`

In [10]:
def test_max_no_improve(data, dataset_name, max_no_improve_values, num_repeats=10):

    results = []
    for max_no_improve in max_no_improve_values:
        for _ in range(num_repeats):
            best_solution, best_cost, best_time = tabu_search(
                data,
                max_iterations=10000,
                max_no_improve=max_no_improve,
                move_type="two_opt",
                time_limit=1000,
                tabu_list_length=10
            )
            # Zbieranie wyników
            results.append({
                "PARAMETR": max_no_improve,
                "WYNIK_1": best_cost,
                "ŚCIEŻKA": best_solution,
                "CZAS": best_time
            })
    # Tworzenie DataFrame z wynikami
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Lista wartości max_no_improve do przetestowania
max_no_improve_values = [10, 50, 150, 250]

# Testowanie dla trzech zestawów danych
df_max_no_improve_1 = test_max_no_improve(data1, "DATA1", max_no_improve_values)
df_max_no_improve_2 = test_max_no_improve(data2, "DATA2", max_no_improve_values)
df_max_no_improve_3 = test_max_no_improve(data3, "DATA3", max_no_improve_values)

# Łączenie wyników w jeden DataFrame
max_no_improve_results = pd.concat([df_max_no_improve_1, df_max_no_improve_2, df_max_no_improve_3], ignore_index=True)

# Zapis wyników do pliku Excel
with pd.ExcelWriter('TS.xlsx', engine='openpyxl', mode='a') as writer:
    max_no_improve_results.to_excel(writer, sheet_name='max_no_improve', index=False)

Iteration 0, Best Cost: 39354.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 20791.00, Elapsed Time: 0.13 seconds
Iteration 20, Best Cost: 14157.00, Elapsed Time: 0.30 seconds
Iteration 30, Best Cost: 12030.00, Elapsed Time: 0.43 seconds
Iteration 40, Best Cost: 11370.00, Elapsed Time: 0.55 seconds
Iteration 50, Best Cost: 11225.00, Elapsed Time: 0.68 seconds
Iteration 60, Best Cost: 11216.00, Elapsed Time: 0.81 seconds
Osiągnięto limit iteracji bez poprawy.
Iteration 0, Best Cost: 49555.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 24789.00, Elapsed Time: 0.14 seconds
Iteration 20, Best Cost: 16077.00, Elapsed Time: 0.27 seconds
Iteration 30, Best Cost: 12315.00, Elapsed Time: 0.43 seconds
Iteration 40, Best Cost: 11403.00, Elapsed Time: 0.56 seconds
Iteration 50, Best Cost: 11360.00, Elapsed Time: 0.70 seconds
Osiągnięto limit iteracji bez poprawy.
Iteration 0, Best Cost: 43635.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 22939.00, Elapsed Time: 0.15 sec

### Badanie wpływu zmian wartości parametru `time_limit`

In [11]:
def test_time_limit(data, dataset_name, time_values, num_repeats=10):

    results = []
    for time_limit in time_values:
        for _ in range(num_repeats):
            best_solution, best_cost, best_time = tabu_search(
                data,
                max_iterations=10000,
                max_no_improve=50,
                move_type="two_opt",
                time_limit=time_limit,
                tabu_list_length=10
            )
            # Zbieranie wyników
            results.append({
                "PARAMETR": time_limit,
                "WYNIK_1": best_cost,
                "ŚCIEŻKA": best_solution,
                "CZAS": best_time
            })
    # Tworzenie DataFrame z wynikami
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Lista wartości time_limit do przetestowania
time_values = [60, 240, 500, 1000]

# Testowanie dla trzech zestawów danych
df_time_limit_1 = test_time_limit(data1, "DATA1", time_values)
df_time_limit_2 = test_time_limit(data2, "DATA2", time_values)
df_time_limit_3 = test_time_limit(data3, "DATA3", time_values)

# Łączenie wyników w jeden DataFrame
time_limit_results = pd.concat([df_time_limit_1, df_time_limit_2, df_time_limit_3], ignore_index=True)

# Zapis wyników do pliku Excel
with pd.ExcelWriter('TS.xlsx', engine='openpyxl', mode='a') as writer:
    time_limit_results.to_excel(writer, sheet_name='time_limit', index=False)

Iteration 0, Best Cost: 42031.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 20888.00, Elapsed Time: 0.12 seconds
Iteration 20, Best Cost: 13522.00, Elapsed Time: 0.23 seconds
Iteration 30, Best Cost: 11455.00, Elapsed Time: 0.34 seconds
Iteration 40, Best Cost: 11163.00, Elapsed Time: 0.50 seconds
Iteration 50, Best Cost: 10995.00, Elapsed Time: 0.61 seconds
Iteration 60, Best Cost: 10995.00, Elapsed Time: 0.72 seconds
Iteration 70, Best Cost: 10995.00, Elapsed Time: 0.83 seconds
Iteration 80, Best Cost: 10995.00, Elapsed Time: 0.94 seconds
Iteration 90, Best Cost: 10995.00, Elapsed Time: 1.05 seconds
Osiągnięto limit iteracji bez poprawy.
Iteration 0, Best Cost: 39686.00, Elapsed Time: 0.01 seconds
Iteration 10, Best Cost: 21413.00, Elapsed Time: 0.14 seconds
Iteration 20, Best Cost: 14089.00, Elapsed Time: 0.26 seconds
Iteration 30, Best Cost: 11806.00, Elapsed Time: 0.37 seconds
Iteration 40, Best Cost: 10946.00, Elapsed Time: 0.48 seconds
Iteration 50, Best Cost: 10892.00